In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
import json
import os

import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer

from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Minhk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Minhk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Minhk\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Minhk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
path = Path('K:/Materials/Statistical Natural Language Processing/Project')
data_path = path/'data'
os.listdir(data_path)

['Home_and_Kitchen.json',
 'Sports_and_Outdoors.csv',
 'Sports_and_Outdoors.json',
 'text_sports.csv']

# Import data

## Sports and Outdoors file

In [3]:
data = [json.loads(line) for line in open(data_path/'Sports_and_Outdoors.json', 'r')]
data[0]

{'_id': {'$oid': '5a132768741a2384e847b8ed'},
 'reviewerID': 'A2PAVURT4NOHE1',
 'asin': '0000031852',
 'reviewerName': 'Leah',
 'helpful': [0, 0],
 'reviewText': 'Bought it for a ballet tutu but it is being worn around the house with leggings. Really cute, full skirt.',
 'overall': 5.0,
 'summary': 'Super cute',
 'unixReviewTime': 1388361600,
 'reviewTime': '12 30, 2013',
 'category': 'Sports_and_Outdoors',
 'class': 1.0}

In [4]:
data = pd.DataFrame(data)
data['_id'] = data['_id'].map(lambda value : value['$oid'])
data.to_csv(data_path/'Sports_and_Outdoors.csv', index='_id')
data

,_id,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,category,class
0,5a132768741a2384e847b8ed,A2PAVURT4NOHE1,0000031852,Leah,"[0, 0]",Bought it for a ballet tutu but it is being wo...,5.0,Super cute,1388361600,"12 30, 2013",Sports_and_Outdoors,1.0
1,5a132768741a2384e847b8e9,A1SNLWGLFXD70K,0000031852,DEVA,"[0, 0]",I origonally didn't get the item I ordered. W...,4.0,Happy with purchase even though it came a lot ...,1392940800,"02 21, 2014",Sports_and_Outdoors,1.0
2,5a132768741a2384e847b8ee,A3URQ0LXLV46E9,0000031852,shortyvee,"[0, 0]",My daughter and her friends love the colors an...,4.0,zebralisous,1400544000,"05 20, 2014",Sports_and_Outdoors,1.0
3,5a132768741a2384e847b8ea,A1KJ4CVG87QW09,0000031852,Donna Carter-Scott,"[0, 0]","Arrived very timely, cute grandbaby loves it. ...",4.0,Cute Tutu,1389657600,"01 14, 2014",Sports_and_Outdoors,1.0
4,5a132768741a2384e847b8eb,AA9ITO6ZLZW6,0000031852,Jazzy77,"[0, 0]",My little girl just loves to wear this tutu be...,5.0,Versatile,1399507200,"05 8, 2014",Sports_and_Outdoors,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3013251,5a1327fe741a2384e879a60d,A1XNL3C5RL8AC,B00LVHLRZ8,Michael Allen Bright,"[0, 0]",This only took 2 days to receive and the cord ...,5.0,Great product.,1405382400,"07 15, 2014",Sports_and_Outdoors,1.0
3013252,5a1327fe741a2384e879a60e,A1BV7953RLM07E,B00LVHLRZ8,Shen,"[0, 0]",Bought this item for travel use and so far it ...,5.0,One of the best Paracord I've used,1405382400,"07 15, 2014",Sports_and_Outdoors,1.0
3013253,5a1327fe741a2384e879a60f,A1ZUS94QF43DAC,B00LVHLRZ8,Sidney Ronson,"[0, 0]",Wasn't expecting this Paracord to be as Vibran...,5.0,Great Strong Paracord!,1405641600,"07 18, 2014",Sports_and_Outdoors,1.0
3013254,5a1327fe741a2384e879a610,A32T585IZ0DJX2,B00LVHLRZ8,wicker558,"[0, 0]",Bought this item for home and so far it was pr...,5.0,Good,1405900800,"07 21, 2014",Sports_and_Outdoors,1.0


# Retrieve only text data

In [5]:
text_data = data[[ 'summary', 'reviewText', 'overall', 'class', 'category']]
test_data = text_data.dropna(axis = 0)
text_data['Text'] = data['summary'] + ". " + data['reviewText']
text_data = text_data[['Text', 'overall', 'category', 'class']]
text_data

C:\Users\Minhk\AppData\Local\Temp/ipykernel_17400/3537793424.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_data['Text'] = data['summary'] + ". " + data['reviewText']


,Text,overall,category,class
0,Super cute. Bought it for a ballet tutu but it...,5.0,Sports_and_Outdoors,1.0
1,Happy with purchase even though it came a lot ...,4.0,Sports_and_Outdoors,1.0
2,zebralisous. My daughter and her friends love ...,4.0,Sports_and_Outdoors,1.0
3,"Cute Tutu. Arrived very timely, cute grandbaby...",4.0,Sports_and_Outdoors,1.0
4,Versatile. My little girl just loves to wear t...,5.0,Sports_and_Outdoors,1.0
...,...,...,...,...
3013251,Great product.. This only took 2 days to recei...,5.0,Sports_and_Outdoors,1.0
3013252,One of the best Paracord I've used. Bought thi...,5.0,Sports_and_Outdoors,1.0
3013253,Great Strong Paracord!. Wasn't expecting this ...,5.0,Sports_and_Outdoors,1.0
3013254,Good. Bought this item for home and so far it ...,5.0,Sports_and_Outdoors,1.0


Concatenate summary with reviewText

In [6]:
X = text_data[['Text', 'overall', 'category']]
y = text_data['class'] #define feature matrix X and labels y 
X_rem, X_sample, y_rem, y_sample = train_test_split(X, y, test_size = 0.005, shuffle = True, stratify=y, random_state = 1234) 
X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size = 0.3, random_state = 1234) #split data 70:30

In [7]:
text_data = pd.DataFrame(X_train)
text_data['class'] = y_train
text_data = text_data.reset_index(drop=True)
text_data

,Text,overall,category,class
0,classy sun shirt. I bought this shirt for my h...,5.0,Sports_and_Outdoors,1.0
1,Regrets. I live in the Caribbean so before buy...,1.0,Sports_and_Outdoors,0.0
2,Worth - May be. This is a very light racket. I...,4.0,Sports_and_Outdoors,1.0
3,Clean and refreshed FAST. There are so many us...,5.0,Sports_and_Outdoors,1.0
4,Triathlete in NEB. I purchased this Pearl Izum...,5.0,Sports_and_Outdoors,1.0
...,...,...,...,...
10541,It's good.... I wear a 34 inch Levi's jean and...,5.0,Sports_and_Outdoors,1.0
10542,Good for the price. This is a descent knife fo...,4.0,Sports_and_Outdoors,1.0
10543,Feels great. Stretches back and hips perfectly...,5.0,Sports_and_Outdoors,1.0
10544,"daily carry. Flawless construction, fit, and f...",5.0,Sports_and_Outdoors,1.0


In [8]:
#Helper function to get lemmatizer to return the original word form
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"V": wordnet.VERB,
                "N": wordnet.NOUN,
                "J": wordnet.ADJ,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize(raw_string):
    string = re.sub(r"'\w?", " ", raw_string)
    string = re.sub(r"[^a-zA-Z \-]", " ", string)
    #tokenize words
    word_tokens = word_tokenize(string)
    #remove stopwords
    stop_words = set(stopwords.words('english') + ['-', 'us'])
    filtered_tokens = [word.lower() for word in word_tokens if not word.lower() in stop_words]
    #stem the remaining words
    lemmatizer = WordNetLemmatizer()
    stemmed_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in filtered_tokens]
    stemmed_string = " ".join(stemmed_tokens)
    return stemmed_string

In [9]:
text_data['Processed Text'] = text_data['Text'].map(lambda raw_string : lemmatize(raw_string))
text_data = text_data[['Text', 'Processed Text', 'overall', 'category', 'class']]
text_data

,Text,Processed Text,overall,category,class
0,classy sun shirt. I bought this shirt for my h...,classy sun shirt bought shirt husband upf love...,5.0,Sports_and_Outdoors,1.0
1,Regrets. I live in the Caribbean so before buy...,regret live caribbean buying product comparati...,1.0,Sports_and_Outdoors,0.0
2,Worth - May be. This is a very light racket. I...,worth may light racket spin speed control woul...,4.0,Sports_and_Outdoors,1.0
3,Clean and refreshed FAST. There are so many us...,clean refresh fast many us amaze soap best cle...,5.0,Sports_and_Outdoors,1.0
4,Triathlete in NEB. I purchased this Pearl Izum...,triathlete neb purchase pearl izumi grip-lite ...,5.0,Sports_and_Outdoors,1.0
...,...,...,...,...,...
10541,It's good.... I wear a 34 inch Levi's jean and...,good wear inch levi jean arge size snug fit wa...,5.0,Sports_and_Outdoors,1.0
10542,Good for the price. This is a descent knife fo...,good price descent knife price son like play w...,4.0,Sports_and_Outdoors,1.0
10543,Feels great. Stretches back and hips perfectly...,feel great stretch back hip perfectly high qua...,5.0,Sports_and_Outdoors,1.0
10544,"daily carry. Flawless construction, fit, and f...",daily carry flawless construction fit finish b...,5.0,Sports_and_Outdoors,1.0


In [12]:
text_data.to_csv(data_path/'text_sports.csv')
text_data[['Processed Text', 'class']].to_csv(data_path/'spam_sports.csv')